# Import Libraries and Define Constants

This cell imports the necessary libraries and defines some constants that will be used throughout the program.

In [ ]:
import cv2
import time
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import threading

print(cv2.__version__)

# Display dimensions
dispW = 640  # Lowered display width for better performance
dispH = 480  # Lowered display height for better performance

# Font settings for FPS display
tPosition = (30, 60)  # Position of the FPS text on the frame
tFont = cv2.FONT_HERSHEY_SIMPLEX  # Font type for the FPS text
tHeight = 1.5  # Font height for the FPS text
tThick = 3  # Font thickness for the FPS text
tColor = (0, 0, 255)  # Font color for the FPS text in BGR format

# Initial HSV values
hueLow, hueHigh = 17, 52
satLow, satHigh = 111, 255
valLow, valHigh = 164, 255
track = 0

# Boolean flag to stop the main loop
stop = False


# Initialize Camera and Sliders
This cell initializes the camera and creates the sliders that will be used to adjust the HSV values and the tracking mode.

In [ ]:
# Initialize the USB camera (usually the first camera is at 0)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

# Create sliders for adjusting HSV values and tracking mode
hue_low_slider = widgets.IntSlider(min=0, max=179, value=hueLow, description='Hue Low')
hue_high_slider = widgets.IntSlider(min=0, max=179, value=hueHigh, description='Hue High')
sat_low_slider = widgets.IntSlider(min=0, max=255, value=satLow, description='Sat Low')
sat_high_slider = widgets.IntSlider(min=0, max=255, value=satHigh, description='Sat High')
val_low_slider = widgets.IntSlider(min=0, max=255, value=valLow, description='Val Low')
val_high_slider = widgets.IntSlider(min=0, max=255, value=valHigh, description='Val High')
track_slider = widgets.IntSlider(min=0, max=1, value=track, description='Track')
stop_button = widgets.Button(description="Stop")


# Define Callback Functions and Display Widgets
This cell defines the callback functions for the sliders and the stop button, and displays the widgets.

In [ ]:
def on_change(change):
    global hueLow, hueHigh, satLow, satHigh, valLow, valHigh, track
    hueLow = hue_low_slider.value
    hueHigh = hue_high_slider.value
    satLow = sat_low_slider.value
    satHigh = sat_high_slider.value
    valLow = val_low_slider.value
    valHigh = val_high_slider.value
    track = track_slider.value

def on_button_click(b):
    global stop
    stop = True

hue_low_slider.observe(on_change, names='value')
hue_high_slider.observe(on_change, names='value')
sat_low_slider.observe(on_change, names='value')
sat_high_slider.observe(on_change, names='value')
val_low_slider.observe(on_change, names='value')
val_high_slider.observe(on_change, names='value')
track_slider.observe(on_change, names='value')
stop_button.on_click(on_button_click)

display(hue_low_slider, hue_high_slider, sat_low_slider, sat_high_slider, val_low_slider, val_high_slider, track_slider, stop_button)


# Define Display Function
This cell defines a function to display the camera feed, mask, and object images using matplotlib

In [ ]:
def display_frame(frame, mask, obj):
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.title('Camera')
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.title('Mask')
    plt.imshow(mask, cmap='gray')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.title('My Object')
    plt.imshow(cv2.cvtColor(obj, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


# Stop Button Thread
Creates a separate thread to monitor the stop button.

In [ ]:
def monitor_stop_button():
    global stop
    while not stop:
        time.sleep(0.01)

# Start the stop button monitoring thread
stop_thread = threading.Thread(target=monitor_stop_button)
stop_thread.start()


# Main Loop
This cell contains the main loop that captures frames from the camera, processes them, and displays them.

In [ ]:
# Initialize FPS and frame count variables
fps = 0
frame_count = 0
display_frequency = 5  # Display every 5 frames to improve performance

# Main loop
while not stop:
    tStart = time.time()
    
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # If frame is read correctly, ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # Resize frame to lower resolution for better performance
    frame = cv2.resize(frame, (dispW, dispH))
    
    # Flip the frame vertically
    frame = cv2.flip(frame, 1)  # Flip around y-axis (horizontal flip)
    
    # Convert frame to HSV color space
    frameHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Put FPS text on the frame
    #cv2.putText(frame, str(int(fps)) + ' FPS', tPosition, tFont, tHeight, tColor, tThick)
    
    # Define HSV color range and create a mask
    lowerBound = np.array([hueLow, satLow, valLow])
    upperBound = np.array([hueHigh, satHigh, valHigh])
    myMask = cv2.inRange(frameHSV, lowerBound, upperBound)
    myMaskSmall = cv2.resize(myMask, (int(dispW/2), int(dispH/2)))
    
    # Apply mask to the frame to extract the object
    myObject = cv2.bitwise_and(frame, frame, mask=myMask)
    myObjectSmall = cv2.resize(myObject, (int(dispW/2), int(dispH/2)))
    
    # Find contours in the mask
    contours, junk = cv2.findContours(myMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        # Sort contours by area and draw the largest one
        contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
        contour = contours[0]
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 3)
    
    # Display the resulting frames using matplotlib every `display_frequency` frames
    frame_count += 1
    if frame_count % display_frequency == 0:
        display_frame(frame, myMaskSmall, myObjectSmall)
    
    # Calculate FPS
    tEnd = time.time()
    loopTime = tEnd - tStart
    fps = 0.9 * fps + 0.1 * (1 / loopTime)

# Release the capture and destroy windows
cap.release()
cv2.destroyAllWindows()

# Ensure the stop thread ends
stop_thread.join()
